In [7]:
import time
import cv2
from datetime import date, timedelta
import mediapipe as mp
import base64
import pymysql
from flask import Flask, render_template, Response
from flask_cors import CORS

# sql 테스트중ㅡㅡ
conn = pymysql.connect(
    user='seeu',
    passwd='12345',
    port=3307,
    host='project-db-stu.ddns.net',
    db='seeu',
    charset='utf8'
)
cursor = conn.cursor()
# ㅡㅡㅡㅡㅡㅡㅡㅡ


app = Flask(__name__)
CORS(app)

@app.route('/')
def index():
    return "대충 인덱스 페이지."


def gen():
    # Set up variables // 변수 설정
    mpDraw = mp.solutions.drawing_utils
    mppose = mp.solutions.pose
    pose = mppose.Pose()

    # Time passed // FPS나 pose를 분석할 frame을 몇 밀리초로 구분할지에 대한 시간 변수 설정
    # Stored as seconds value * 100
    time_passed = 0
    start_time = time.time()

    # Each frame is an array
    # Inside that array there are tuples (constant arrays basically) 
    # Each tuple is a (x, y) coordinate
    positions = []

    # Counter for how many frames have been processed
    counter = 0
    cap = cv2.VideoCapture(0)
    ptime = 0

    # Read until video is completed // 비디오 읽는 부분
    while(cap.isOpened()):
      # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Search for pose
            results = pose.process(frameRGB)
            if results.pose_landmarks:
                # Draw the lines
                mpDraw.draw_landmarks(frame, results.pose_landmarks, mppose.POSE_CONNECTIONS)

                # Convert results into array
                lms = []


                for id, lm in enumerate(results.pose_landmarks.landmark):
                    h, w, c = frame.shape
                    cx, cy = int(lm.x*w), int(lm.y * h)
                    lms.append((cx, cy))
                positions.append(lms)

                # Write Coordinates of left ear to file
                with open('leftEar.txt', 'a') as f:
                    f.write(str(positions[counter][7]))
                    f.write("\n")

                # Write Coordinates of right ear to file
                with open('rightEar.txt', 'a') as f:
                    f.write(str(positions[counter][8]))
                    f.write("\n")

                counter+=1

            # Calculate FPS
            ctime = time.time()
            fps = int(1/(ctime-ptime))
            ptime = ctime

            # Time Logic
            time_passed = int((time.time() - start_time)*100)

            left_pure_diff = 0
            # Trigger every 5 1/100 seconds
            if time_passed % 5 == 0:
                # Left Leg coordinate - left ear coordinate
                if counter>0:
                    left_pure_diff = positions[counter-1][27][1] - positions[counter-1][7][1]
                    if left_pure_diff>-100 and left_pure_diff<100:
                        print(f"{left_pure_diff} : 넘어짐, 쓰러짐 추정!!")
                        cv2.putText(frame, 'abnormal havior detection', (10, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
                        now = datetime.now()
                        current_time = f'{now.year}{now.month}{now.day}{now.hour}{now.minute}{now.second}{now.microsecond}'
                        print(now)
                        cv2.imwrite(f'{current_time}.png', frame)
                        # 실험중인 코드(base64 문자열 처리)
                        jpg_img = cv2.imencode('.jpg', frame)
                        b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')
                        print(f'// 이거 그 값 : {b64_string[:5]}')
                        
                        # sql문으로 base64 문자열 처리된 이미지 코드 삽입
                        sql = "INSERT INTO tbl_video (v_cam,v_url,v_capture) VALUES (%s, %s, %s);"
                        cursor.execute(sql, ("CAM_12345","ee.ee",b64_string))
                        conn.commit()
                        # sql 문 테스트중 ㅡㅡㅡㅡㅡㅡㅡ
                        
                        
            frame = cv2.resize(frame, (0,0), fx=1.5, fy=1.5) 
            frame = cv2.imencode('.jpg', frame)[1].tobytes()
            yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            time.sleep(0.1)
        else: 
            # sql 문 테스트중 ㅡㅡㅡㅡㅡ
            conn.close()
            break


@app.route("/user/<uid>")
def api_info(uid):
    # 파라미터로 가져오는 url생성하기
    sql = "SELECT * from tbl_senior;"
    cursor.execute(sql, ("CAM_12345","ee.ee",b64_string))
    conn.commit()
    return uid
        
@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an frame tag."""
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')



if __name__=="__main__":
    app.run(host="59.0.236.92",port="9000")
    
    app = Flask(__name__)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://59.0.236.92:9000/ (Press CTRL+C to quit)
59.0.236.92 - - [29/Jul/2022 17:16:53] "GET / HTTP/1.1" 200 -
59.0.236.92 - - [29/Jul/2022 17:17:00] "GET /video_feed HTTP/1.1" 200 -
59.0.236.92 - - [29/Jul/2022 17:17:14] "GET /video_feed HTTP/1.1" 200 -
Error on request:
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\serving.py", line 319, in run_wsgi
    execute(self.server.app)
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\serving.py", line 310, in execute
    for data in application_iter:
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\wsgi.py", line 462, in __next__
    return self._next()
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\wrappers\response.py", line 49, in _iter_encoded
    for item in iterable:
  File "C:\Users\smhrd\AppData\Local\Temp/ipykernel_13104/4160911797.py", line 105, in gen
    
NameError: name 'datetime' is not defined


78 : 넘어짐, 쓰러짐 추정!!


59.0.236.92 - - [29/Jul/2022 17:21:47] "GET /video_feed HTTP/1.1" 200 -
59.0.236.92 - - [29/Jul/2022 17:22:35] "GET /video_feed HTTP/1.1" 200 -
Error on request:
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\serving.py", line 319, in run_wsgi
    execute(self.server.app)
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\serving.py", line 310, in execute
    for data in application_iter:
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\wsgi.py", line 462, in __next__
    return self._next()
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\wrappers\response.py", line 49, in _iter_encoded
    for item in iterable:
  File "C:\Users\smhrd\AppData\Local\Temp/ipykernel_13104/4160911797.py", line 105, in gen
    
NameError: name 'datetime' is not defined


70 : 넘어짐, 쓰러짐 추정!!


...

84 : 넘어짐, 쓰러짐 추정!!


59.0.236.92 - - [29/Jul/2022 17:24:58] "GET /video_feed HTTP/1.1" 200 -
Error on request:
Traceback (most recent call last):
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\serving.py", line 319, in run_wsgi
    execute(self.server.app)
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\serving.py", line 310, in execute
    for data in application_iter:
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\wsgi.py", line 462, in __next__
    return self._next()
  File "C:\Users\smhrd\anaconda3\Lib\site-packages\werkzeug\wrappers\response.py", line 49, in _iter_encoded
    for item in iterable:
  File "C:\Users\smhrd\AppData\Local\Temp/ipykernel_13104/4160911797.py", line 105, in gen
    
NameError: name 'datetime' is not defined


65 : 넘어짐, 쓰러짐 추정!!


59.0.236.92 - - [29/Jul/2022 17:31:31] "GET /video_feed HTTP/1.1" 200 -
192.241.214.18 - - [29/Jul/2022 18:52:42] code 400, message Bad request syntax ('MGLNDD_59.0.236.92_9000')
192.241.214.18 - - [29/Jul/2022 18:52:42] "MGLNDD_59.0.236.92_9000" HTTPStatus.BAD_REQUEST -
66.240.236.119 - - [29/Jul/2022 23:12:25] code 400, message Bad request version ('\x03á¥(\x06(\x00\x00°À0À,À(À$À\x14À')
66.240.236.119 - - [29/Jul/2022 23:12:25] "  ü~à4)Éiú{r,½S3Ú#Ðá¥((  °À0À,À(À$ÀÀ" HTTPStatus.BAD_REQUEST -
66.240.236.119 - - [29/Jul/2022 23:12:29] "GET / HTTP/1.1" 200 -
34.78.6.216 - - [29/Jul/2022 23:12:35] "GET / HTTP/1.1" 200 -
66.240.236.119 - - [29/Jul/2022 23:12:49] code 400, message Bad request syntax ('\x03Ìêãb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

66.240.236.119 - - [29/Jul/2022 23:12:49] "Ìêãb                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        